# DRACO - Milestone 2: Dataset exploration

This document is structured as follow:

1. Characters Data - Extraction and Processing
2. Movie Data - Extraction and Processing
3. Actors Ethinicites - Exploration

---

In [247]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [248]:
DATA_FOLDER = './Data'

MOVIES_SUMMARIES_FOLDER = DATA_FOLDER + '/MovieSummaries'
CHARACTER_PATH = MOVIES_SUMMARIES_FOLDER + '/character.metadata.tsv'
MOVIE_PATH = MOVIES_SUMMARIES_FOLDER + '/movie.metadata.tsv'
NAME_PATH = MOVIES_SUMMARIES_FOLDER + 'name.clusters.txt'
PLOT_PATH = MOVIES_SUMMARIES_FOLDER + 'plot_summaries.txt'


ETHNICITY_PATH = DATA_FOLDER + '/ethnicities_data.tsv'
ETHNICITY_GROUP_PATH = DATA_FOLDER + '/ethnicity_group_data.tsv'

KAGGLE_PATH = DATA_FOLDER + '/Kaggle'
MOVIES_KAGGLE_PATH = KAGGLE_PATH + '/movies_metadata.csv'
RATING_PATH = KAGGLE_PATH + '/ratings.csv'


## Importation of each datasets

### **CMU Movie corpus dataset**

### Character data

In [249]:
characters_original = pd.read_csv(CHARACTER_PATH, sep='\t', header=None, 
    names = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie release date", "Character name", "Birth", 
    "Gender", "Height", "Ethnicity ID", "Name", "Age at movie release",
    "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"])

### CMU Movie data

In [250]:
movies_CMU = pd.read_csv(MOVIE_PATH, sep='\t', header=None, 
    names = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie name","Movie release date", "Box office revenue","Movie runtime","Movie language","Movie countries","Movie genres" ])

###  **External dataset**

### Ethnicity data

These data were created using the files `Ethnicity_extraction.iypnb` and `Ethnicity_grouped.iypnb`

In [251]:
ethnicities_original = pd.read_csv(ETHNICITY_PATH, sep='\t',  
                               header=0, names=["Ethnicity ID", "Ethnicity"])

ethnic_group_original = pd.read_csv(ETHNICITY_GROUP_PATH, sep='\t')

### Kaggle dataset

To expand our dataset with additional information, we decided to include the full MovieLens Dataset found on [Kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data). This dataset contain the following files
- **movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv:** Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv:** Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

- **links.csv:** The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

- **links_small.csv:** Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv:** The subset of 100,000 ratings from 700 users on 9,000 movies.

In [252]:
movies_kaggle = pd.read_csv(MOVIES_KAGGLE_PATH, header = 0,
    names = ["Adult", "Belongs to collection", "Budget", "Genres", "Homepage", 
    "Kaggle_ID", "IMBD_ID", "Language", "Orignal title", "Plot summary",
    "Popularity", "Poster path", "Production companies","Production countries",
    "Movie release date","Box office revenue","Movie runtime","Spoken language","status",
    "Tagline", "Movie name", "Video", "Vote average", "Vote count"])

C:\Users\ovola\AppData\Local\Temp\ipykernel_1720\2743342987.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_kaggle = pd.read_csv(MOVIES_KAGGLE_PATH, header = 0,


## Preprocessing of the Data

### Cleaning the data

#### Character data

Let's see what the movie dataset looks like.

In [253]:
characters_original.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie release date,Character name,Birth,Gender,Height,Ethnicity ID,Name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [254]:
characters_original.describe()

,Wikipedia Movie ID,Height,Age at movie release
count,4.506690e+05,154824.000000,292556.000000
mean,1.396975e+07,1.788893,37.788523
std,1.079662e+07,4.379940,20.587870
min,3.300000e+02,0.610000,-7896.000000
25%,3.759292e+06,1.676400,28.000000
50%,1.189065e+07,1.750000,36.000000
75%,2.366501e+07,1.830000,47.000000
max,3.750192e+07,510.000000,103.000000


As wee can see there is issues in the dataset. 
- First some ethinicites as well as gender are not specified.
- Second we can see that the age at movie release spans from -7896 to 103 years.

For our analysis we have decided to discard all characters that don't have a specified ethnicity as well as a specified gender. Moreover, only strictly positive ages will be taken into account. Also, since we are not interrested in the height and name of the actor that the column `Freebase Movie ID` is redundant with `Wikipedia Movie ID`, we have decided to drop those columns. 

In [255]:
characters = characters_original.copy()
characters = characters.drop(['Freebase Movie ID','Height','Name'], axis=1)

In [256]:
characters = characters[characters['Ethnicity ID'].notna()]
characters = characters[characters['Gender'].notna()]
characters = characters[characters['Age at movie release'] > 0]

In [257]:
characters.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
1,975900,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,/m/044038p,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
5,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft
11,975900,2001-08-24,Tres,1959-03-09,M,/m/064b9n,42.0,/m/0bgchrs,/m/0bgchrw,/m/03ydsb
27,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp


#### Movie data

##### **CMU dataset**

Let's see what the CMU movie dataset looks like.

In [258]:
movies_CMU.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Box office revenue,Movie runtime,Movie language,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


All columns of this dataset are relevant for the following justifications:
- Wikipedia Movie ID: used to get website request
- Freebase Movie ID: used to merge the plot summaries to the MCU movie dataset
- Movie name: used to merge other external movie dataset with MCU movie dataset
- Box office revenue

**Need to be finished**

Then let's do few first step of pre-processing related to the dataset (the remaining pre-processing will be done later after merging the datasets). We first need to correct the format of columns which countains dictionaries, e.g `Movie language`, `Movie countries` and `Movie genres`. In our case, it would be much more convenient to have lists instead. Let's process it accordingly.

In [259]:
movies_CMU["Movie countries"] = movies_CMU["Movie countries"].apply(lambda x: list(json.loads(x).values()) if len(json.loads(x).values()) > 0 else 'NaN')
movies_CMU["Movie genres"] = movies_CMU["Movie genres"].apply(lambda x: list(json.loads(x).values()) if len(json.loads(x).values()) > 0 else 'NaN')
movies_CMU["Movie language"] = movies_CMU["Movie language"].apply(lambda x: list(json.loads(x).values()) if len(json.loads(x).values()) > 0 else 'NaN')

As the dataset was realesed on 2013, the data from this year are not complet and thus should be removed. To simplify this step we can add a category for the year of release in addition of the date of release.

In [260]:
#Errors = 'coerce' will force the values that are outside the bound to be NaT
movies_CMU["Movie release year"] = pd.to_datetime(movies_CMU["Movie release date"],format='mixed',errors = 'coerce').dt.year
#Remove the NaN and NaT values
movies_CMU = movies_CMU[movies_CMU["Movie release year"].notna()]
#Express all years of realese as int.
movies_CMU["Movie release year"] = movies_CMU["Movie release year"].astype("int")

#Finaly let's remove the movies released in 2013
movies_CMU = movies_CMU[movies_CMU['Movie release year'] < 2013]


#### **Kaggle dataset**

As explain before, to expand our dataset with more information, we are going to use the Kaggle dataset. Let's then see what can of informations could be relevant for our project.

In [261]:
movies_kaggle.head()

,Adult,Belongs to collection,Budget,Genres,Homepage,Kaggle_ID,IMBD_ID,Language,Orignal title,Plot summary,...,Movie release date,Box office revenue,Movie runtime,Spoken language,status,Tagline,Movie name,Video,Vote average,Vote count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Among these different columns, the informations relevant to analyze the success depending on the cast ethnociyt of a movie are : `Budget`,`Plot summary`, `Movie release date`, `Box office revenue`, `Movie runtime`, `Movie name`, `Vote average` and `Vote count`
Thus let's drop of the other unnecessary columns.

In [262]:
movies_kaggle = movies_kaggle.drop(["Adult", "Belongs to collection",  "Genres", "Homepage", 
                                    "Kaggle_ID", "IMBD_ID", "Language", "Orignal title", 
                                    "Popularity", "Poster path", "Production companies","Production countries",
                                    "Spoken language","status", "Tagline", "Video"],axis=1)

As for the CMU, let's also change the movie release date format by adding the year of release instead.

In [263]:
#Errors = 'coerce' will force the values that are outside the bound to be NaT
movies_kaggle["Movie release year"] = pd.to_datetime(movies_kaggle["Movie release date"],format='mixed',errors = 'coerce').dt.year
#Remove the NaN and NaT values
movies_kaggle = movies_kaggle[movies_kaggle["Movie release year"].notna()]
#Express all years of realese as int.
movies_kaggle["Movie release year"] = movies_kaggle["Movie release year"].astype("int")


In [264]:
movies_kaggle.head()

,Budget,Plot summary,Movie release date,Box office revenue,Movie runtime,Movie name,Vote average,Vote count,Movie release year
0,30000000,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995
1,65000000,When siblings Judy and Peter discover an encha...,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995
2,0,A family wedding reignites the ancient feud be...,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,1995
3,16000000,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,1995
4,0,Just when George Banks has recovered from his ...,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,1995


#### **Merging the dataset**

For better efficiency, we're going to merge the 2 datasets toghether and clean them after this step.
Before merging let's have a general idea of the size of each datasets and the number of values that can be used directly(not Nan values). This is important to know how many data we could loss if we decided to keep informations present in only one dataset.

In [265]:
#Print the datasets size
print("Lenght of the MCU dataset : " ,movies_CMU.shape[0])
print("Lenght of the Kaggle dataset : " ,movies_kaggle.shape[0])

Lenght of the MCU dataset :  74650
Lenght of the Kaggle dataset :  45376


In [266]:
#Print the sum of not Nan values 
print(f"Sum of not Nan values in the MCU dataset:\n {movies_CMU.notna().sum()}")
print(f"Sum of not Nan values in the Kaggle dataset :\n {movies_kaggle.notna().sum()}")

Sum of not Nan values in the MCU dataset:
 Wikipedia Movie ID    74650
Freebase Movie ID     74650
Movie name            74650
Movie release date    74650
Box office revenue     8327
Movie runtime         58608
Movie language        74650
Movie countries       74650
Movie genres          74650
Movie release year    74650
dtype: int64
Sum of not Nan values in the Kaggle dataset :
 Budget                45376
Plot summary          44435
Movie release date    45376
Box office revenue    45376
Movie runtime         45130
Movie name            45376
Vote average          45376
Vote count            45376
Movie release year    45376
dtype: int64


Eventhought the MCU dataset has higher number of missing values, it also has the higher number of values. Therefore it should be used for merging using the left join. However the left join should be only done on the `Movie name` and `Movie release date` to let the possibility of using the values from the Kaggle dataset to complete the `Nan` values of the CMU dataset. We choose the 2 columns to ensure of the identity of the movie, as we can have multiple movies with the same name.
Concerning the general smaller size of the Kaggle dataset, we should also keep in mind that if we want to analyze the vote scores of the merged dataset, we're going to loss half or more of the MCU dataset. But a deeper feasibility analysis will be perform after the data cleaning.

In [267]:
#Let's merge the data
movies = movies_CMU.copy().merge(movies_kaggle,on=['Movie name','Movie release date'],how='left')
#Vizualise the new mereged dataset
movies.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Box office revenue_x,Movie runtime_x,Movie language,Movie countries,Movie genres,Movie release year_x,Budget,Plot summary,Box office revenue_y,Movie runtime_y,Vote average,Vote count,Movie release year_y
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001,28000000,Melanie Ballard (Natasha Henstridge) is a hard...,14010832.0,98.0,4.8,299.0,2001.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",1988,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama],1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On the merged dataset, we have 2 columns for the `Box office revenue` and the `Movie runtime`. As the Kaggle dataset contains more information concerning the box office than the MCU dataset, we should used its information to complete the missing values of the MCU one. Same thing for the `Movie runtime`. <br>
Plus we should also only keep the year of release.

In [268]:
#Let's replace the Nan values of the MCU dataset by the one of the Kaggel dataset
movies['Box office revenue'] = movies['Box office revenue_x'].fillna(movies['Box office revenue_y'])
movies['Movie runtime'] = movies['Movie runtime_x'].fillna(movies['Movie runtime_y'])

#Let's drop the Box office revenue, movie runtime and also the date of release
movies = movies.drop(['Box office revenue_x','Box office revenue_y','Movie runtime_x','Movie runtime_y'],axis=1)

After this, let's see how changed the sum of available data on the merged dataset.

In [269]:
#Print the sum of not Nan values 
print(f"Sum of not Nan values in the MCU dataset:\n {movies_CMU.notna().sum()}")
print(f"Sum of not Nan values of the merged dataset:\n {movies.notna().sum()}")

Sum of not Nan values in the MCU dataset:
 Wikipedia Movie ID    74650
Freebase Movie ID     74650
Movie name            74650
Movie release date    74650
Box office revenue     8327
Movie runtime         58608
Movie language        74650
Movie countries       74650
Movie genres          74650
Movie release year    74650
dtype: int64
Sum of not Nan values of the merged dataset:
 Wikipedia Movie ID      74660
Freebase Movie ID       74660
Movie name              74660
Movie release date      74660
Movie language          74660
Movie countries         74660
Movie genres            74660
Movie release year_x    74660
Budget                   8434
Plot summary             8406
Vote average             8434
Vote count               8434
Movie release year_y     8434
Box office revenue      13488
Movie runtime           58815
dtype: int64


Thanks to the Kaggle dataset we went from 8401 values for the budget to 13572. Plus it also seems that this dataset contains 10 more movies than the MCU one.

To always have the maximal number of available values, we're going to remove the `Nan` values only when the analysis we want to perfom need it. 
However to perform an analysis which is relevant to our subject, there are some pre-processing process to do. We want to focus on movies that could potentially be found on TV or cinema and where real actors are playing in. <br> It means that we should keep movies of duration greater than 10min but smaller than smaller than 4 hours (240min). <br>


In [270]:
#Let's remove the movies of the wrong direction
movies = movies[movies['Movie runtime'] >= 10]
movies = movies[movies['Movie runtime'] < 240]

Plus we should also pay attention to the movie of the Animation field as the actors that have played inside are rather voice actors than actors. 

In [271]:
# First let's see all the unique genres, to discard the animation ones:
unique_genres = set()
movies['Movie genres'].apply(lambda x: unique_genres.update(x))
print(unique_genres)

{'Anti-war', 'Western', 'New Queer Cinema', 'Action', 'History', 'Sword and sorcery', 'Revisionist Fairy Tale', 'Interpersonal Relationships', 'Animation', 'Juvenile Delinquency Film', 'Anti-war film', 'Latino', 'Comedy of Errors', 'Comedy film', 'Fantasy Comedy', 'Hip hop movies', 'Cult', 'Chick flick', 'Cyberpunk', 'Bruceploitation', 'Parkour in popular culture', 'Illnesses & Disabilities', 'Alien Film', 'Travel', 'Gay pornography', 'Zombie Film', 'Education', 'Gothic Film', 'Romantic fantasy', 'Swashbuckler films', 'Feature film', 'Kafkaesque', 'Propaganda film', 'Revisionist Western', 'Expressionism', 'Buddy cop', 'Coming-of-age film', 'Slice of life story', 'Family Film', 'Epic Western', "Children's", 'Music', 'Americana', 'Animated cartoon', 'Coming of age', 'Political cinema', 'Archives and records', 'Auto racing', 'Horror Comedy', 'Caper story', 'Period Horror', 'World cinema', 'Extreme Sports', 'Instrumental Music', 'Roadshow/Carny', 'Gay Interest', 'Clay animation', "Children

In [272]:
values_to_find = ['Anime', 'Animation', 'Computer Animation', 'Clay animation', 'Animated cartoon','Stop motion']
movies = movies[movies['Movie genres'].apply(lambda x: not(any(value in x for value in values_to_find)))]

Let' see how is movies dataset after pre-processing it.

In [273]:
movies.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie language,Movie countries,Movie genres,Movie release year_x,Budget,Plot summary,Vote average,Vote count,Movie release year_y,Box office revenue,Movie runtime
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001,28000000,Melanie Ballard (Natasha Henstridge) is a hard...,4.8,299.0,2001.0,14010832.0,98.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",2000,NaN,NaN,NaN,NaN,NaN,NaN,95.0
2,28463795,/m/0crgdbh,Brun bitter,1988,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",1988,NaN,NaN,NaN,NaN,NaN,NaN,83.0
3,9363483,/m/0285_cd,White Of The Eye,1987,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",1987,NaN,NaN,NaN,NaN,NaN,NaN,110.0
4,261236,/m/01mrr1,A Woman in Flames,1983,[German Language],[Germany],[Drama],1983,NaN,NaN,NaN,NaN,NaN,NaN,106.0


In [274]:
print(f"Sum of not Nan values of the merged dataset:\n {movies.notna().sum()}")

Sum of not Nan values of the merged dataset:
 Wikipedia Movie ID      55598
Freebase Movie ID       55598
Movie name              55598
Movie release date      55598
Movie language          55598
Movie countries         55598
Movie genres            55598
Movie release year_x    55598
Budget                   8047
Plot summary             8021
Vote average             8047
Vote count               8047
Movie release year_y     8047
Box office revenue      12793
Movie runtime           55598
dtype: int64


Appart maybe from the box office revenue all the other categories seems to be sufficiants.

#### Ethnicity data

Let's see what the ethnicity dataset looks like.

In [275]:
ethnicities_original.head()

,Ethnicity ID,Ethnicity
0,/m/044038p,NaN
1,/m/0x67,African Americans
2,/m/064b9n,Omaha people
3,/m/041rx,Jewish people
4,/m/033tf_,Irish Americans


Here again we have NaN values for some ethnicities, we can drop them now.

In [276]:
ethnicities = ethnicities_original.copy()
ethnicities = ethnicities[ethnicities['Ethnicity'].notna()]

In [277]:
ethnicities.head()

,Ethnicity ID,Ethnicity
1,/m/0x67,African Americans
2,/m/064b9n,Omaha people
3,/m/041rx,Jewish people
4,/m/033tf_,Irish Americans
5,/m/04gfy7,Indian Americans


#### Grouping ethnicities in ethnic groups 

In [278]:
nb_ethnicities = len(np.unique(ethnicities['Ethnicity']))
print('Number of ethnicities:',nb_ethnicities)

Number of ethnicities: 431


As we can see we have a lot of different ethnicities in the dataframe. For our analysis it could be interresting to group them according to [UK's list of ethnic groups ](https://www.ethnicity-facts-figures.service.gov.uk/style-guide/ethnic-groups). This list groups ethnicity in four main groups:
- White
- Black, Caribbean or African
- Asian, Middle East and Tribes
- Mixed or multiple ethnic groups

Moreover to have clusters of similar size, we have decided to combine `Asian or Asian British` with `Other ethnic group` to a larger group called `Asian, Middle East and Tribes`.

In [279]:
ethnic_group = ethnic_group_original.copy()
ethnic_group.head()

,Ethnicity,Ethnic Group
0,Aceh,"Asian, Middle East and Tribes"
1,Afghans in India,"Asian, Middle East and Tribes"
2,Agrawal,"Asian, Middle East and Tribes"
3,Asian people,"Asian, Middle East and Tribes"
4,Bengali Brahmins,"Asian, Middle East and Tribes"


### Merging datafames

Now that all the datasets are cleaned, we can merge them in one big dataframe.

Let's first merge the character dataset with the list of ethnicities, so we can add their ethinicity to the dataset

In [280]:
characters_extended = characters.copy().merge(ethnicities, how='inner', on='Ethnicity ID')
characters_extended.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans
2,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp,African Americans
3,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans
4,156558,2001-06-27,Jody,1978-12-30,M,/m/0x67,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans


Let's merge the characters with the list of ethnic groups to obtain add the ethnic group of each actor

In [281]:
characters_extended = characters_extended.merge(ethnic_group, how='inner', on='Ethnicity')
characters_extended.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity,Ethnic Group
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans,"Black, Caribbean or African"
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans,"Black, Caribbean or African"
2,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp,African Americans,"Black, Caribbean or African"
3,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans,"Black, Caribbean or African"
4,156558,2001-06-27,Jody,1978-12-30,M,/m/0x67,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans,"Black, Caribbean or African"


Then let's merge the character dataset with the movie dataset to link the actors with the movies they have played


In [282]:
characters_movies = characters_extended.copy().merge(movies, how='inner', on=['Wikipedia Movie ID','Movie release date'])
characters_movies.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,...,Movie countries,Movie genres,Movie release year_x,Budget,Plot summary,Vote average,Vote count,Movie release year_y,Box office revenue,Movie runtime
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,...,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001,28000000,Melanie Ballard (Natasha Henstridge) is a hard...,4.8,299.0,2001.0,14010832.0,98.0
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,...,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001,28000000,Melanie Ballard (Natasha Henstridge) is a hard...,4.8,299.0,2001.0,14010832.0,98.0
2,975900,2001-08-24,Tres,1959-03-09,M,/m/064b9n,42.0,/m/0bgchrs,/m/0bgchrw,/m/03ydsb,...,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001,28000000,Melanie Ballard (Natasha Henstridge) is a hard...,4.8,299.0,2001.0,14010832.0,98.0
3,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp,...,[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",2000,NaN,NaN,NaN,NaN,NaN,NaN,95.0
4,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,...,[United States of America],"[Crime Fiction, Drama, Coming of age]",2001,16000000,"The story of Jody, a misguided, 20-year-old Af...",7.3,57.0,2001.0,29381649.0,123.0


### Showing the feasability of the project 

In this section, we are only interested in the number of different films, actors and characters we have in the processed dataframe. This is to determine whether the dataframe contains enough data to be analyzed.

In [283]:
print(f"Sum of not Nan values of the merged dataset:\n {characters_movies.notna().sum()}")

Sum of not Nan values of the merged dataset:
 Wikipedia Movie ID                 74843
Movie release date                 74843
Character name                     47090
Birth                              74843
Gender                             74843
Ethnicity ID                       74843
Age at movie release               74843
Freebase character/actor map ID    74843
Freebase character ID              47090
Freebase actor ID                  74843
Ethnicity                          74843
Ethnic Group                       74843
Freebase Movie ID                  74843
Movie name                         74843
Movie language                     74843
Movie countries                    74843
Movie genres                       74843
Movie release year_x               74843
Budget                             20292
Plot summary                       20276
Vote average                       20292
Vote count                         20292
Movie release year_y               20292
Box office 

##### On the actor side 

In [284]:
nb_actors = len(np.unique(characters_movies['Freebase actor ID']))
print("Number of actors:",nb_actors)
nb_characters = np.shape(characters_movies)[0]
print("Number of characters:",nb_characters)
#Print the number of actors on each of the categories
characters_movies.drop_duplicates(subset='Freebase actor ID')['Ethnic Group'].value_counts()

Number of actors: 6756
Number of characters: 74843


Ethnic Group
White                              2547
Asian, Middle East and Tribes      1914
Black, Caribbean or African        1347
Mixed or multiple ethnic groups     948
Name: count, dtype: int64

##### On the movie side 

In [285]:
unique_movies_id = np.unique(characters_movies['Wikipedia Movie ID'])
print("Number of movies:",len(unique_movies_id))
print("Sum of not Nan values of the merged dataset:\n",
       characters_movies[characters_movies['Wikipedia Movie ID'].isin(unique_movies_id)].notna().sum())

Number of movies: 26203
Sum of not Nan values of the merged dataset:
 Wikipedia Movie ID                 74843
Movie release date                 74843
Character name                     47090
Birth                              74843
Gender                             74843
Ethnicity ID                       74843
Age at movie release               74843
Freebase character/actor map ID    74843
Freebase character ID              47090
Freebase actor ID                  74843
Ethnicity                          74843
Ethnic Group                       74843
Freebase Movie ID                  74843
Movie name                         74843
Movie language                     74843
Movie countries                    74843
Movie genres                       74843
Movie release year_x               74843
Budget                             20292
Plot summary                       20276
Vote average                       20292
Vote count                         20292
Movie release year_y        

**There's a mistake here (we should have 26203 movies) but I don't know why**